In [11]:
using System;
using System.Collections.Generic;
using System.Linq;

// Enum для статусов заказов
public enum OrderStatus //определяются перечисления
{
    Created,
    InProgress,
    Completed
}

public class Customer //заказчик
{
    public string FirstName { get; set; } //имя заказчика
    public string LastName { get; set; } //фамилия
    public string PhoneNumber { get; set; } // номер телефона

    public Customer(string firstName, string lastName, string phoneNumber) //добавления заказчика
    {
        FirstName = firstName;
        LastName = lastName;
        PhoneNumber = phoneNumber;
    }
}

public class Employee //сотрудник
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string Position { get; set; } //должность
    public int OrdersProcessed { get; private set; } //кол. обработанных зака

    public Employee(string firstName, string lastName, string position)
    {
        FirstName = firstName;
        LastName = lastName;
        Position = position;
        OrdersProcessed = 0;
    }

    public void ProcessOrder()
    {
        OrdersProcessed++;
    }
}


public class Order //заказ
{
    public Guid OrderId { get; } //индефикатор
    public string Description { get; set; } //описание заказа
    public Employee AssignedEmployee { get; set; } //ответственый сотрудник
    public Customer Customer { get; set; }
    public OrderStatus Status { get; private set; }
    public DateTime CreatedAt { get; } //дата создания

    public Order(string description, Customer customer)
    {
        OrderId = Guid.NewGuid(); //генерация уникального номера
        Description = description;
        Customer = customer;
        Status = OrderStatus.Created;
        CreatedAt = DateTime.Now;
    }
//событие
    public event Action<Order> StatusChanged; //событие
    public void UpdateStatus(OrderStatus newStatus)
    {
        Status = newStatus; //Изменяет статус заказа на указанный новый статус
        StatusChanged?.Invoke(this); //Вызывает все подписчики события StatusChanged, передавая им сам объект заказа.
    }
}

//StatusChanged позволяет другим компонентам автоматически реагировать на изменения статуса заказа без прямого взаимодействия с объектом заказа.
//? - Если ни один подписчик не зарегистрирован, метод просто пропустит вызов.

public class Company
{
    public string Name { get; set; }
    public List<Employee> Employees { get; } = new List<Employee>(); //список сотрудников
    public List<Order> Orders { get; } = new List<Order>(); //список заказов

    public Company(string name)
    {
        Name = name;
    }

    public void AddEmployee(Employee employee) //Добавление сотрудника
    {
        Employees.Add(employee);
    }

    public void AddOrder(Order order) //Добавление заказа
    {
        Orders.Add(order);
    }

    public void AssignEmployeeToOrder(Order order, Employee employee) //Назначение сотрудника на заказ
    {
        order.AssignedEmployee = employee;
    }

    public List<Order> GetOrdersByStatus(OrderStatus status) //Получение заказов по статусу
    {
        return Orders.Where(o => o.Status == status).ToList(); //Фильтрует список заказов по заданному статусу.
    }

    public List<Order> GetOrdersByCustomer(Customer customer) //Получение заказов по клиенту
    {
        return Orders.Where(o => o.Customer == customer).ToList(); //Фильтрует список заказов по конкретному клиенту.
    }

    public void GenerateEmployeeReport() //Генерация отчета о сотрудниках
    {
        Console.WriteLine("Employee Report:");
        foreach (var employee in Employees)
        {
            Console.WriteLine($"- {employee.FirstName} {employee.LastName}: {employee.OrdersProcessed} orders processed");
        }
    }
}
//Выводит отчет о всех сотрудниках компании, показывая их имена, фамилии и количество обработанных ими заказов.

var company = new Company("Аптайм");
var emp1 = new Employee("Валерия", "Шананина", "Крутой разрабочик");
var emp2 = new Employee("Анастасия", "Косишнева", "Подает пиццу");
company.AddEmployee(emp1);
company.AddEmployee(emp2);

var customer1 = new Customer("Аделя", "Шаукерова", "123-456-7890");
var customer2 = new Customer("Егорка", "Поляниский", "098-765-4321");

var order1 = new Order("Дизайн сайта", customer1);
var order2 = new Order("База данный сайта", customer2);

order1.StatusChanged += order =>
{
    Console.WriteLine($"Заказ: {order.OrderId} ({order.Description}) изменил статус на {order.Status}");
};

order2.StatusChanged += order =>
{
    Console.WriteLine($"Заказ: {order.OrderId} ({order.Description}) теперь {order.Status}");
};

company.AddOrder(order1);
company.AddOrder(order2);

company.AssignEmployeeToOrder(order1, emp1);
company.AssignEmployeeToOrder(order2, emp2);

order1.UpdateStatus(OrderStatus.InProgress);
emp1.ProcessOrder();
order2.UpdateStatus(OrderStatus.Completed);
emp2.ProcessOrder();
company.GenerateEmployeeReport();

Console.WriteLine("\nЗаказы в работе:");
foreach (var order in company.GetOrdersByStatus(OrderStatus.InProgress))
{
    Console.WriteLine($"Заказ🔧 {order.OrderId}: {order.Description} - В работе у {order.AssignedEmployee?.FirstName ?? "неизвестного"}");
}

Console.WriteLine("\nСтатус заказов:");
foreach (var order in company.Orders)
{
    if (order.Status == OrderStatus.Completed)
        Console.WriteLine($"✅ {order.OrderId}: {order.Description} - Окончено");
    else if (order.Status == OrderStatus.InProgress)
        Console.WriteLine($"⏳ {order.OrderId}: {order.Description} - В процессе");
    else
        Console.WriteLine($"❓ {order.OrderId}: {order.Description} - Создан");
}


Заказ: 7bf92c0f-568c-4287-b960-f6c3cd299635 (Дизайн сайта) изменил статус на InProgress
Заказ: 8d29d3f8-6594-4be7-b113-c6e66a1053d8 (База данный сайта) теперь Completed
Employee Report:
- Валерия Шананина: 1 orders processed
- Анастасия Косишнева: 1 orders processed

Заказы в работе:
Заказ🔧 7bf92c0f-568c-4287-b960-f6c3cd299635: Дизайн сайта - В работе у Валерия

Статус заказов:
⏳ 7bf92c0f-568c-4287-b960-f6c3cd299635: Дизайн сайта - В процессе
✅ 8d29d3f8-6594-4be7-b113-c6e66a1053d8: База данный сайта - Окончено
